In [39]:
# =========================
# Importar librerías
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [40]:
# =========================
# Config
# =========================
INPUT_CSV   = "Ventas_totales_limpio.csv"
OUTPUT_IQR  = "Ventas_sin_outliers_IQR.csv"
OUTPUT_STD  = "Ventas_sin_outliers_STD.csv"

# Parámetros
IQR_K       = 1.5   # 1.5 IQR clásico
Z_THRESHOLD = 3.0   # 3 desviaciones estándar

# Guardar gráficas
GUARDAR_BOXPLOTS = True
CARPETA_PLOT_INI = "Boxplots_Antes"
CARPETA_PLOT_IQR = "Boxplots_Después_IQR"
CARPETA_PLOT_STD = "Boxplots_Después_STD"

In [41]:
# =========================
# 1) Carga de datos
# =========================
df = pd.read_csv(INPUT_CSV)

# Parseo de fecha si existe la columna
if "indice_tiempo" in df.columns:
    df["indice_tiempo"] = pd.to_datetime(df["indice_tiempo"], dayfirst=True, errors="coerce")

# Nos quedamos con columnas numéricas para detección de outliers
cols_num = df.select_dtypes(include="number").columns.tolist()
print(f"Filas: {len(df)}  |  Columnas numéricas: {len(cols_num)}")
print(cols_num)

Filas: 65  |  Columnas numéricas: 23
['ventas_precios_corrientes', 'ventas_precios_constantes', 'ventas_totales_canal_venta', 'salon_ventas', 'canales_on_line', 'ventas_totales_medio_pago', 'efectivo', 'tarjetas_debito', 'tarjetas_credito', 'otros_medios', 'ventas_totales_grupo_articulos', 'subtotal_ventas_alimentos_bebidas', 'bebidas', 'almacen', 'panaderia', 'lacteos', 'carnes', 'verduleria_fruteria', 'alimentos_preparados_rotiseria', 'articulos_limpieza_perfumeria', 'indumentaria_calzado_textiles_hogar', 'electronicos_articulos_hogar', 'otros']


In [42]:
# =========================
# Utilidad: guardar boxplots por columna
# =========================
def guardar_boxplots_por_columna(data, columnas, carpeta, titulo_prefijo=""):
    if not GUARDAR_BOXPLOTS:
        return
    Path(carpeta).mkdir(parents=True, exist_ok=True)
    for c in columnas:
        plt.figure()
        data.boxplot(column=c)
        plt.title(f"{titulo_prefijo}{c}")
        plt.tight_layout()
        plt.savefig(Path(carpeta) / f"boxplot_{c}.png", dpi=150)
        plt.close()

In [43]:
# =========================
# 2) Boxplots iniciales
# =========================
guardar_boxplots_por_columna(df, cols_num, CARPETA_PLOT_INI, "Antes - ")

In [44]:
# =========================
# 3) Método IQR (Rango Intercuartílico)
# =========================
mask_keep_iqr = pd.Series(True, index=df.index)
resumen_iqr = []

for c in cols_num:
    q1 = df[c].quantile(0.25)
    q3 = df[c].quantile(0.75)
    iqr = q3 - q1
    low = q1 - IQR_K * iqr
    high = q3 + IQR_K * iqr
    col_mask = df[c].between(low, high, inclusive="both")
    mask_keep_iqr &= col_mask
    resumen_iqr.append({
        "columna": c,
        "q1": q1, "q3": q3, "iqr": iqr,
        "low": low, "high": high,
        "outliers_en_columna": int((~col_mask).sum())
    })

df_iqr = df[mask_keep_iqr].copy()
df_iqr.to_csv(OUTPUT_IQR, index=False)

print("\n=== IQR ===")
print(f"Filas después de IQR: {len(df_iqr)} (eliminadas: {len(df)-len(df_iqr)})")
print(pd.DataFrame(resumen_iqr).sort_values("outliers_en_columna", ascending=False).head(10))

guardar_boxplots_por_columna(df_iqr, cols_num, CARPETA_PLOT_IQR, "Después IQR - ")


=== IQR ===
Filas después de IQR: 55 (eliminadas: 10)
                                columna            q1            q3  \
20  indumentaria_calzado_textiles_hogar  1.010425e+06  2.206737e+06   
11    subtotal_ventas_alimentos_bebidas  3.558672e+07  6.717124e+07   
21         electronicos_articulos_hogar  2.169127e+06  6.042114e+06   
18       alimentos_preparados_rotiseria  4.379606e+05  8.814768e+05   
14                            panaderia  1.412394e+06  2.973192e+06   
3                          salon_ventas  4.391419e+07  9.452455e+07   
1             ventas_precios_constantes  2.370200e+04  2.699670e+04   
22                                otros  3.313479e+06  7.255727e+06   
8                      tarjetas_credito  1.362389e+07  3.371681e+07   
15                              lacteos  3.996896e+06  1.037440e+07   

             iqr           low          high  outliers_en_columna  
20  1.196312e+06 -7.840429e+05  4.001206e+06                    4  
11  3.158451e+07 -1.179005e

In [45]:
# =========================
# 4) Método Desviación Estándar
# =========================
mask_keep_std = pd.Series(True, index=df.index)
resumen_std = []

for c in cols_num:
    mean = df[c].mean()
    std  = df[c].std(ddof=0)
    if std == 0 or np.isnan(std):
        col_mask = pd.Series(True, index=df.index)  # sin variación → no se filtra por esta col
        outliers_count = 0
        low = high = np.nan
    else:
        z = (df[c] - mean) / std
        col_mask = z.abs() <= Z_THRESHOLD
        outliers_count = int((~col_mask).sum())
        low = mean - Z_THRESHOLD * std
        high = mean + Z_THRESHOLD * std

    mask_keep_std &= col_mask
    resumen_std.append({
        "columna": c,
        "mean": mean, "std": std,
        "low": low, "high": high,
        "outliers_en_columna": outliers_count
    })

df_std = df[mask_keep_std].copy()
df_std.to_csv(OUTPUT_STD, index=False)

print("\n=== Desviación Estándar ===")
print(f"Filas después de STD: {len(df_std)} (eliminadas: {len(df)-len(df_std)})")
print(pd.DataFrame(resumen_std).sort_values("outliers_en_columna", ascending=False).head(10))

guardar_boxplots_por_columna(df_std, cols_num, CARPETA_PLOT_STD, "Después STD - ")


=== Desviación Estándar ===
Filas después de STD: 61 (eliminadas: 4)
                                columna          mean           std  \
1             ventas_precios_constantes  2.606370e+04  4.617139e+03   
22                                otros  5.776004e+06  3.194767e+06   
12                              bebidas  8.878731e+06  5.823245e+06   
20  indumentaria_calzado_textiles_hogar  1.832572e+06  1.072509e+06   
3                          salon_ventas  7.583609e+07  4.027672e+07   
0             ventas_precios_corrientes  7.224020e+04  4.699177e+04   
2            ventas_totales_canal_venta  7.373840e+07  4.422329e+07   
6                              efectivo  2.378445e+07  1.261091e+07   
5             ventas_totales_medio_pago  7.423803e+07  4.356599e+07   
4                       canales_on_line  2.077902e+06  1.834110e+06   

             low          high  outliers_en_columna  
1   1.221228e+04  3.991511e+04                    2  
22 -3.808299e+06  1.536031e+07          

In [46]:
# =========================
# 5) Resumen final
# =========================
print("\nCSV generados:")
print(f" - {OUTPUT_IQR}")
print(f" - {OUTPUT_STD}")
if GUARDAR_BOXPLOTS:
    print("\nBoxplots guardados en:")
    print(f" - {CARPETA_PLOT_INI}/")
    print(f" - {CARPETA_PLOT_IQR}/")
    print(f" - {CARPETA_PLOT_STD}/")


CSV generados:
 - Ventas_sin_outliers_IQR.csv
 - Ventas_sin_outliers_STD.csv

Boxplots guardados en:
 - Boxplots_Antes/
 - Boxplots_Después_IQR/
 - Boxplots_Después_STD/
